# First write to DB

In [45]:
import polars as pl
import re 
import time
def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]


In [46]:
news = pl.read_csv(   
    'news.csv',
    dtypes={"content_hash": pl.UInt64},     
)
newsner = pl.read_csv(   
    'newsner.csv',
    dtypes={"content_hash": pl.UInt64},     
)

In [47]:
news.head(1)

index,topic,date_extract,date_article,content,portal,link,link_sim_score,title,summary,summary_llm,summary_sim_score,authors,state,city,content_hash,content_nchar,tit_summary,tit_sum_embed
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,u64,str,str,str


In [4]:
news = news.with_columns(
    pl.concat_str(
        [
            pl.col('state'),
            pl.col('city'),
            pl.col("title"),
            pl.col("summary_llm"),
        ],
        separator=" ",
    ).alias("tit_summary"),
)

In [8]:
news.head(3)

index,topic,date_extract,date_article,content,portal,link,link_sim_score,title,summary,summary_llm,summary_sim_score,authors,state,city,content_hash,content_nchar,tit_summary
i64,str,str,str,str,str,str,f64,str,str,str,f64,str,str,str,u64,i64,str
1,"""narcotráfico""","""2023-08-16""","""2023-08-16""","""Seguinos Por L…","""https://www.ar…","""https://www.la…",0.561929,"""Condenaron a s…","""Brian está det…","""Seis condenado…",0.5237,"""n-a""","""Santa Fe""","""Rosario""",9259959111029284149,4543,"""Santa Fe Rosar…"
2,"""narcotráfico""","""2023-08-16""","""2023-08-16""","""Escuadrón 1 “R…","""https://www.ar…","""http://www.dia…",0.476911,"""Encuentran 32 …","""na""","""La Fiscalía di…",0.5592,"""Diario El Oran…","""Salta""","""Orán""",6046667906668190206,707,"""Salta Orán Enc…"
3,"""narcotráfico""","""2023-08-16""","""2023-08-16""",""". Compartir en…","""https://www.ar…","""https://www.ar…",0.526558,"""Prefectura sec…","""Efectivos de l…","""Puerto Rico in…",0.5686,"""n-a""","""Argentina""","""Argentina""",12244461015022324118,657,"""Argentina Arge…"


## Embedings

In [5]:
text  = news['tit_summary'].to_list()[0]
text

'Santa Fe Rosario Condenaron a seis personas por venta de drogas, entre ellos un sobrino del "Rengo" Insaurralde Seis condenados por tráfico de estupefacientes en un proceso abreviado. El Tribunal Oral Federal 3 afronta una tercera secuencia del accionar de la banda'

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(text)

In [7]:
len(embeddings)

384

In [9]:
type(embeddings)

numpy.ndarray

In [16]:
def process_dataframe(df):
    # Load the transformer model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Function to apply the model to each element in 'tit_summary'
    def embed_text(summary):
        try:
            embeddings = model.encode(summary)
            return embeddings.tolist()
        except Exception as e:
            print(f"An error occurred while processing text: {summary}. Error: {str(e)}")
            return None # or return a default value if desired

    # Create a new column 'tit_sum_embed' with the embeddings of 'tit_summary'
    embeddings_col = df['tit_summary'].apply(embed_text, return_dtype=pl.Object)
    new_df = df.with_columns(embeddings_col.rename('tit_sum_embed'))

    return new_df


In [17]:
news = process_dataframe(news)

In [27]:
df = news.drop('tit_sum_embed')

In [28]:
from sentence_transformers import SentenceTransformer
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq

def process_dataframe(df):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    def embed_text(summary):
        try:
            embeddings = model.encode(summary)
            return embeddings.tolist() # Convert numpy array to list
        except Exception as e:
            print(f"An error occurred while processing text: {summary}. Error: {str(e)}")
            return None

    embeddings_col = df['tit_summary'].apply(embed_text, return_dtype=pl.Object)
    new_df = df.with_columns(embeddings_col.rename('tit_sum_embed'))

    return new_df

# Assuming `df` is your original DataFrame
processed_df = process_dataframe(df)

# Convert the Polars DataFrame to a PyArrow Table
arrow_table = pa.Table.from_pandas(processed_df.to_pandas())

# Write the PyArrow Table to a Parquet file
pq.write_table(arrow_table, 'news.parquet')


In [30]:
import pyarrow.parquet as pq
import polars as pl

def read_parquet_to_polars(file_path):
    # Read the Parquet file using PyArrow
    arrow_table = pq.read_table(file_path)
    
    # Convert the PyArrow Table to a Pandas DataFrame
    pandas_df = arrow_table.to_pandas()
    
    # Convert the Pandas DataFrame to a Polars DataFrame
    polars_df = pl.from_pandas(pandas_df)

    return polars_df

# Usage example
file_path = 'news.parquet'
data = read_parquet_to_polars(file_path)
